<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# Administration: Validate user profiles

* 👟 Ready To Run!
* 🗃️ Administration
* 👤 User Management

__Requirements__
* 🔒 Administrator Privileges

Some organizations require member profiles to contain values beyond the minumum required attributes necessary to create a valid user in an ArcGIS Online Organization or ArcGIS Enterprise. For instance, in order to comply with policies and regulations, an organization may require a profile picture or a brief description. This notebook will check attribute values for users of the organization to monitor profiles.

To get started, let's import the necessary libraries and connect to our GIS.

In [1]:
import os
import datetime as dt

import pandas as pd

from arcgis.gis import GIS, ItemPropertie, ItemTypeEnum

In [2]:
gis = GIS("home")

Then, let's create a list containing strings that represent all the values that your organization requires (beyond the required attributes for a valid user profile).

`access_type` refers to the `User.access` property of a user and it indicates the level of access of the user: private, org, or public.

While `access_type` may not be a requirement for your organization to validate a user profile, we have added it here to demonstrate how other useful properties can also be extracted for each user profile to learn more about them. 

In [3]:
complete_profile = ['description', 'thumbnail', 'access', 'access_type']

We'll define a function that loops through the list we created and inspects a user object for the values of these attributes. We'll create a list of True/False values for each user regarding the necessary attributes.

In [4]:
def get_missing_profile_attrs(member):
    non_compliance = []
    for attr in complete_profile:
        if attr=='access_type':
            non_compliance.append(member.access)
        else:
            if getattr(member, attr) == None:    
                non_compliance.append(False)
            else:
                non_compliance.append(True)
    return non_compliance

Now we can create a list of users in the GIS and loop through them, calling the function written above to inspect each user object. This will create a dictionary with usernames as the _key_ and the list of True/False status for the required attributes as _values_.

In [5]:
user_profile_status = {}
for user in gis.users.search("NOT esri_*"):
    missing_profile_atts = get_missing_profile_attrs(user)
    user_profile_status[user.username] = missing_profile_atts

The pandas library can be used to create a dataframe from the above dictionary.

In [6]:
user_profile_df = pd.DataFrame(data=user_profile_status, index=complete_profile).T
user_profile_df.head()

,description,thumbnail,access,access_type
achapkowski_geosaurus,False,False,True,org
amani_geosaurus,True,True,True,public
andrew57,True,True,True,private
api_data_owner,False,False,True,org
arcgis_python,True,True,True,public


This dataframe can then be written to a _.csv._ file on your fileshare,

In [7]:
output_dir = "/arcgis/home/"
current = str(int(dt.datetime.now().timestamp()))
out_file = "output_org_user_profile" + "_" + current + ".csv"

user_profile_df.to_csv(os.path.join(output_dir, out_file), index_label='username')

and the dataframe can be written to a _.csv_ file item on your Organization.

In [8]:
item_props = ItemProperties(
    "title" = out_file,
    "type" = ItemTypeEnum.CSV,
    "tags" = "user_profile_report",
    "snippet" = "Report on user profile data from Python API"
)

root_folder = gis.content.folders.get()

gis.content.add(
    item_properties = item_props,
    file= os.path.join(output_dir + out_file)
).result()

<Item title:"output_org_user_profile_1686349431" type:CSV owner:MMajumdar_geosaurus>

You may download this item if you wish, and if you decide to delete this item after having used it, you may run the script below by updating the `item_id` with the id of this file in your organization.

In [ ]:
item = gis.content.get(item_id)
item.delete()

# Conclusion

This notebook checked attribute values for an organization's users and wrote the results to a _.csv_ file. This file can then be analyzed to validate that all user profiles contain the minimum required attributes as defined by any policies or regulations.